# Ego4D create enhanced NLQ dataset for pre-training

## Download Data and Setup Environment

### **Fill In Your Access Info Here**
If you don't have access and secret keys, first sign the Ego4D License at [ego4ddataset.com](https://ego4ddataset.com)

In [1]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""

### **Set up CLIs and Download Annotations + Repo**

In [2]:
# Download the AWS and Ego4D CLIs, then download the annotations locally
# Set up the AWS CLI
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip -o awscliv2.zip >/dev/null
!sudo ./aws/install >/dev/null 2>&1
!aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID" && aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY"
!rm "awscliv2.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.9M  100 57.9M    0     0   140M      0 --:--:-- --:--:-- --:--:--  141M


### Install the ego4d CLI and Download Data

In [3]:
# Set up the Ego4D CLI
!pip install ego4d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00
  Created wheel for ego4d: filename=ego4d-1.7.3-py3-none-any.whl size=118250 sha256=8dcf40d85e7f181e0f6324d12d50d9605f7ee4fbb11783c1b779d921718b5a63
  Stored in directory: /root/.cache/pip/wheels/65/a8/89/a6187e3bc9a85e81899ab8d5ddc2011c9954d3b6cb84d47e03
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha25

In [4]:
# Download the Ego4D Annotations to ego4d_data/
!ego4d --output_directory="/content/ego4d_data/" --datasets annotations --benchmarks nlq -y --version v1

Datasets to download: {'annotations'}
Download Path: /content/ego4d_data/v1
Ego4D Metadata: /content/ego4d_data/ego4d.json
Checking requested datasets and versions...
Created download directory for version 'v1' of dataset: 'annotations' at: /content/ego4d_data/v1/annotations
Benchmarks specified but ignored without a benchmarks field in manifest.
Retrieving object metadata from S3...
100% 31/31 [00:00<00:00, 793.93object/s]
Checking if latest file versions are already downloaded...
  0% 0/31 [00:00<?, ?file/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: ego4d-consortium-sharing.s3.us-west-1.amazonaws.com. Connection pool size: 10
 32% 10/31 [00:00<00:00, 39.30file/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: ego4d-consortium-sharing.s3.us-west-1.amazonaws.com. Connection pool size: 10
100% 31/31 [00:00<00:00, 105.04file/s]
No existing videos to filter.
 99% 2.49G/2.51G [00:24<00:00, 204MiB/s]Checking file integrity..

Load the json files in pyton dictionaries

In [6]:
import json
metadata = json.load(open("/content/ego4d_data/ego4d.json"))
narration = json.load(open("/content/ego4d_data/v1/annotations/narration.json"))
nlq_val = json.load(open("/content/ego4d_data/v1/annotations/nlq_val.json"))

In [ ]:
# Open a file in write mode
with open("UIDs_of_visual_features.txt", "w") as file:
    for item in features_list:
        file.write(f"{item}\n")

print("List saved")

In [7]:
# Open the file in read mode
with open("UIDs_of_visual_features.txt", "r") as file:
    vis_features_from_txt = [line.strip() for line in file]

print(vis_features_from_txt[0:12])

['d66f42bb-822b-444a-bce0-ddd15b29bd1b', '._e091d046-ebbf-456e-a56d-5aa743c88202', '._46f82b66-39e2-46e1-beee-229848800b1b', '._bd8fd228-ec64-4515-b7ab-af261c0a5807', '2e1f40e1-169e-488c-bd9b-9c47cfee31ab', '._80535887-f8d0-4b40-aef3-47a3912b0bcb', 'da8b599a-c7a4-45aa-9a4c-a12af1c6926a', '._459503f6-d3b1-4641-843c-63bf1644e6da', '._1f2e985f-ea94-4ec4-bc22-f5516e693d7d', '892a25e2-df0c-4577-af57-b720e73654f6', '._a1cf9215-730c-4c5a-9980-3108250984c2', '._6e8d8309-821a-4b0a-8e3f-6cd74f59cb44']


In [8]:
features_list = vis_features_from_txt

In [9]:
videos = metadata.get("videos")
clips = metadata.get("clips")

Skip the videos that are included in nlq_train and nlq_val

In [10]:
nlq_val_videos = nlq_val.get("videos")
nlq_val_videos_ids= []
for item in nlq_val_videos:
  nlq_val_videos_ids.append(item["video_uid"])

print("Number of nlq_val_videos_ids: ",len(nlq_val_videos_ids))

Number of nlq_val_videos_ids:  247


In [11]:
nlq_train = json.load(open("/content/ego4d_data/v1/annotations/nlq_train.json"))

nlq_train_videos = nlq_train.get("videos")
nlq_train_videos_ids= []
for item in nlq_train_videos:
  nlq_train_videos_ids.append(item["video_uid"])

print("Number of nlq_train_videos_ids:", len(nlq_train_videos_ids))

Number of nlq_train_videos_ids: 754


In [12]:
metadata_clips = metadata.get("clips")
metadata_clips_video_ids = []
for item in metadata_clips:
  metadata_clips_video_ids.append(item["video_uid"])

# of videos included in metadata
print(len(metadata_clips_video_ids))

12283


# In the above code there is duplicates. only unique videos needed


In [13]:
unique_video_ids = list(set(metadata_clips_video_ids))

# Print the number of unique video IDs,
# means only 3878 video has clips
print("number of videos with clips:", len(unique_video_ids))

number of videos with clips: 3878


In [14]:
narration_videos = list(narration.keys())
print(len(narration_videos))

9645


In [15]:
excluded_videos = nlq_train_videos_ids + nlq_val_videos

allowed_videos = [item for item in narration_videos if item not in excluded_videos]
print("allowed_videos:", len(allowed_videos))

allowed_videos: 8891


So far, we used only narration.json. Now, we will combine the informations from the metadata and narrations to create real json file

In [16]:
# Convert the lists to sets
set1 = set(features_list)
set2 = set(allowed_videos)

# Find the intersection of the two sets
common_files = set1.intersection(set2)

# Get the count of the common files
common_files_count = len(common_files)

# Print the count
print("Number of common files:", common_files_count)

common_files = list(common_files)

Number of common files: 8839


# Creation of main dataset

* This script processes video metadata and narration data to build a dictionary of videos, each containing clips and their corresponding annotations, while applying filters on the number of narrations and clips.


In [18]:
def get_narrations_for_current_clip(video_start_sec, video_end_sec, narrations_dict):
  """
  This method takes as input the start_sec, the end_sec of the clip and
  all narrations from narration.json that correspond to the video.

  The method outputs a filtered narrations_dict containing only the narrations inside the video

  I do it this way because later is very easy to change the logic of the strategy
  when we already have only the narrations from the current list
  """

  LENGTH = len(narrations_dict)


  result_list = []

  i = 0

  # We iterate through the narrations until we find the first narration for the clip
  while LENGTH> i+1 and narrations_dict[i]['timestamp_sec']< video_start_sec:
    i = i + 1

  start_index = i

  #We iterate through the remaining part, create and add dictionaries to the list
  #until we find the last narration that is fully inside the clip
  while LENGTH> i+1 and narrations_dict[i+1]['timestamp_sec']< video_end_sec:
    i = i + 1

  end_index = i

  return narrations_dict[start_index:end_index+1]

In [19]:
def get_language_queries(narrs, pos, num_of_narrs, start_sec):
  language_narrations = []

  if pos+num_of_narrs >= len(narrs):
    print(pos)
    print(num_of_narrs)
    print(len(narrs))

  narrs_start_sec = narrs[pos]["timestamp_sec"] - start_sec
  narrs_end_sec = narrs[pos+num_of_narrs +1]["timestamp_sec"] - start_sec

  for narr in narrs[position: position + num_of_narrs]:
    language_narrations.append({
            "clip_start_sec" : narrs_start_sec ,
            "clip_end_sec" : narrs_end_sec,
            "query": narr["narration_text"]
        })

  return language_narrations

we will get the narrations from here and then we apply the random stuff, and selection of the narrations, directly from the narrations for the current video

In [20]:
import random
random.seed(12)

# all videos 1 clip 8-12 consecutive narrs

In [21]:
#Set parameters
min_narr_per_clip = 11  # min number of narrations per clip
max_narr_per_clip = 14 # man number of narrations per clip
filter_max_clips = 1   # max used number of clips per video


metadata_clips_as_dict = metadata.get("clips")
big_dictionary = {"videos":[]}
videos_counter = 0
skipped_videos = 0

for video_uid in narration:
  # Skip videos not in common_files
  if video_uid not in common_files:
    skipped_videos = skipped_videos+1
    continue

  # get clips for this video
  metadata_clips = [item for item in metadata_clips_as_dict if item.get("video_uid") == video_uid]
  if len(metadata_clips) == 0:
    continue

  list_of_clips = []
  # here we filter out how many clips we are going to use, starting from the first clip
  for clip in metadata_clips[0:filter_max_clips]:
    dic_with_metadata = {
        "clip_uid" : clip["clip_uid"],
        "video_start_sec" : clip["video_start_sec"],
        "video_end_sec" : clip["video_end_sec"],
        "annotations":[]
    }

    video = narration[video_uid]
    if not video:
      continue
    narrator_1 = video.get("narration_pass_1")
    if not narrator_1:
      continue

    narrations_for_vid = (narrator_1.get("narrations"))
    if len(narrations_for_vid) < 1:
      continue

    # Generate a random number corresponding to the count of narrations for the current clip, for example 12
    number_of_narrations = random.randint(min_narr_per_clip, max_narr_per_clip)

    # Get all narrations for the current clip in the correct format
    narrations_for_clip = get_narrations_for_current_clip(clip["video_start_sec"], clip["video_end_sec"], narrations_for_vid)

    if len(narrations_for_clip) <= number_of_narrations +2:
      continue

    position = random.randint(0, len(narrations_for_clip) - number_of_narrations - 2)

    language_queries = get_language_queries(narrations_for_clip, position, number_of_narrations, clip["video_start_sec"])

    if len(language_queries) < 1:
      continue

    annotation_uid = narrations_for_vid[0]["annotation_uid"]
    dic_with_metadata["annotations"].append({
        "language_queries":language_queries,
        "annotation_uid":annotation_uid
        })


    list_of_clips.append(dic_with_metadata)

  # Append video to final dictionary if it has valid clips
  if len(list_of_clips) != 0:
    ids_dictionary = {
        "video_uid" : video_uid,
        "clips": list_of_clips
    }
    big_dictionary["videos"].append(ids_dictionary)
  videos_counter = videos_counter + 1

print("Total number of videos:", videos_counter)

Total number of videos: 3123


In [22]:
random.shuffle(big_dictionary["videos"])

## Check if everything is correct

In [23]:
number_of_queries_per_video = []
number_of_queries_per_clip = []
number_of_queries_per_annotation = []
anns = []
total_number_of_queries = 0

for vid in big_dictionary["videos"]:

    number_of_queries_v = 0
    for clip in vid["clips"]:

        number_of_queries_c = 0
        for ann in clip["annotations"]:

            num_of_q_ann = 0
            for query in ann["language_queries"]:
                total_number_of_queries = total_number_of_queries + 1
                num_of_q_ann = num_of_q_ann +1
                number_of_queries_c = number_of_queries_c +1
                number_of_queries_v = number_of_queries_v +1
            number_of_queries_per_annotation.append(num_of_q_ann)
        number_of_queries_per_clip.append(number_of_queries_c)
    number_of_queries_per_video.append(number_of_queries_v)

print("Total number of queries:", total_number_of_queries)
print("number of clips with 0 language queries", number_of_queries_per_clip.count(0))

Total number of queries: 26040
number of clips with 0 language queries 0


## Saving the dataset

In [26]:
# Save the modified dictionary back to the JSON file
with open('consecutive.json', 'w') as file:
    json.dump(big_dictionary, file, indent=4)

In [25]:
consecutive_merged = {"videos": big_dictionary["videos"] + nlq_train["videos"]}
random.shuffle(consecutive_merged["videos"])

In [27]:
# Save the modified dictionary back to the JSON file
with open('consecutive_merged.json', 'w') as file:
    json.dump(consecutive_merged, file, indent=4)